In [11]:

from unstructured.partition.pdf import partition_pdf
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings as fe
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings as pe
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [12]:
openai_api_key = {put_your_openai_api_key_here}

In [13]:
prompt_template = """
You are financial analyst tasking with providing bank informations.
Answer the question based only on the following context, which can include text and tables:
{context}
Question: {question}
Just return the helpful answer in as much as detailed possible.
Answer:
"""

In [14]:
# loading faiss db
embeddings = fe(openai_api_key=openai_api_key)
# db = FAISS.load_local("faiss_index", embeddings)
db = FAISS.load_local("bank_data", embeddings)

In [15]:
qa_chain = LLMChain(llm=ChatOpenAI(model="gpt-4", openai_api_key = openai_api_key, max_tokens=1024),
                        prompt=PromptTemplate.from_template(prompt_template))

In [16]:
def answer(question):
    relevant_docs = db.similarity_search(question)
    context = ""
    for d in relevant_docs:
        if d.metadata['type'] == 'text':
            context += '[text]' + d.metadata['original_content']
        elif d.metadata['type'] == 'table':
            context += '[table]' + d.metadata['original_content']
        
    result = qa_chain.invoke({'context': context, 'question':  RunnablePassthrough()})
    return result

In [17]:
result =answer("tell me about how to apply house loans")

In [18]:
result['text']

"Smart Bank offers five types of loan schemes:\n\n1. Overdraft Facilities: This is intended to manage cash flow at competitive interest rates. It's linked to a current account and allows users to use as much as they need up to their limit. The limit can be increased or decreased through internet banking. Interest is paid only on the amount used, not the full limit.\n\n2. Housing Loans: This includes several schemes such as purchasing land to construct a house, purchasing a house or partly constructed house, completing construction/renovation/extension/repair of an existing house, and construction of houses/apartments for sale. The maximum loan amount is Rs. 50 Million with a repayment period of 25 years. Eligibility includes being a Sri Lankan citizen above 18 years, should not be a defaulter of Smart Bank or any other financial institution, and can include professionals, permanent employees of government, private sector, self-employed, farmers, cultivators etc.\n\nThe rates of advance